In [ ]:
import numpy as np
import pandas as pd
from itertools import product
import matplotlib.pyplot as plt
from scipy.stats import norm

Best so far: "!V:VarTransform=P,D:NTrees=4000:BoostType=Grad:Shrinkage=0.1:MaxDepth=3:SeparationType=GiniIndex:nCuts=20:UseRandomisedTrees:UseNvars=8:UseBaggedBoost:BaggedSampleFraction=0.6:PruneMethod=CostComplexity:PruneStrength=60"

A lot of trees > 3000
play with:
    BoostType
    shrinkage
    maxdepth
    ncuts
    usenvars
    baggedsamplefraction

RMS test: 8.00

20497 trees: 10.583min: 32 trees/sec

In [ ]:
def getModels(numberModels = 10, letter = "A"):
    models = []
    commands = []
    ntrees = 0
    for modNumber in range(numberModels):
        s = "factory.BookMethod(dataloader, TMVA::Types::kBDT, "
        #name = "BDTG_" + letter + "_" + str(modNumber)
        name = 'Form("BDTG_' + letter + '_' + str(modNumber) + '_%d", testSet)'
        options = "!V"
        #varChoice = np.random.choice(getVarTransform(maxSymbols=2))
        varChoice = getVarTransformChoice(maxSymbols=4)
        if varChoice != "":
            options += ":VarTransform=" + varChoice
        t = getNTreesChoice(1000, 3000)
        ntrees += t
        options += ":NTrees=" + str(t)
        #options += ":NTrees=" + "500"
        options += ":BoostType=" + np.random.choice(getBoostType())
        #options += ":BoostType=" + "Grad"
        options += ":Shrinkage=" + str(getShrinkageChoice(0.03, 0.70))
        #options += ":Shrinkage=" + "0.1"
        options += ":MaxDepth=" + str(getMaxDepthChoice(4, 9))
        #options += ":MaxDepth=" + "3"
        options += ":SeparationType=" + np.random.choice(getSeparationType())
        #options += ":SeparationType=" + "GiniIndex"
        options += ":nCuts=" + str(getNCutsChoice(20, 100))
        #options += ":nCuts=" + "20"
        options += ":UseRandomisedTrees=F"
        options += ":UseNvars=" + str(getUseNvarsChoice(20, 100))
        #options += ":UseNvars=" + "8"
        options += ":UseBaggedBoost"
        options += ":BaggedSampleFraction=" + str(getBaggedSampleFractionChoice(1.0, 5.0))
        #options += ":BaggedSampleFraction=" + "0.6"
        options += ":PruneMethod=" + np.random.choice(getPruneMethod())
        options += ":PruneStrength=" + str(getPruneStrengthChoice(0, 100))
        options += ":PruningValFraction=" + str(getPruningValFractionChoice(0.0, 2.0))
        models.append(s + name + ",\n\t\"" + options + "\");")
        commands.append("nohup bash autotrain.sh \"phi\" 0 \"BDTG_" + letter + "_" + str(modNumber) + "\" \"" + options + "\" &")
        commands.append("nohup bash autotrain.sh \"phi\" 1 \"BDTG_" + letter + "_" + str(modNumber) + "\" \"" + options + "\" &")
        commands.append("nohup bash autotrain.sh \"phi\" 2 \"BDTG_" + letter + "_" + str(modNumber) + "\" \"" + options + "\" &")
    print(ntrees)
    return models, commands


def getModelsVar(a):
    models = []
    ntrees = getBaggedSampleFraction(spacing=11)
    for modNumber in range(len(ntrees)):
        s = "factory.BookMethod(dataloader, TMVA::Types::kBDT, "
        name = "BDTG_" + str(modNumber)
        options = "!V"
        #varChoice = np.random.choice(getBestVarTransform(5))
        varChoice = "P,N,D"
        if varChoice != "":
            options += ":VarTransform=" + varChoice
        #options += ":NTrees=" + str(np.random.choice(getNTrees()))
        options += ":NTrees=" + "100"
        #options += ":BoostType=" + np.random.choice(getBoostType())
        options += ":BoostType=" + "Grad"
        #options += ":Shrinkage=" + str(np.random.choice(getShrinkage()))
        options += ":Shrinkage=" + "0.32"
        #options += ":MaxDepth=" + str(np.random.choice(getMaxDepth()))
        options += ":MaxDepth=" + "6"
        #options += ":SeparationType=" + np.random.choice(getSeparationType())
        options += ":SeparationType=" + "GiniIndex"
        #options += ":nCuts=" + str(np.random.choice(getNCuts()))
        options += ":nCuts=" + "30"
        options += ":UseRandomisedTrees"
        #options += ":UseNvars=" + str(np.random.choice(getUseNvars()))
        options += ":UseNvars=" + "20"
        options += ":UseBaggedBoost"
        #options += ":BaggedSampleFraction=" + str(np.random.choice(getBaggedSampleFraction()))
        options += ":BaggedSampleFraction=" + str(ntrees[modNumber])
        options += ":PruneMethod=NoPruning"
        models.append(s + "\"" + name + "\",\n\t\"" + options + "\");")
    return models


def getVarTransform(minSymbols=0, maxSymbols=2):
    varTransform = ["P", "G", "D", "N"]
    good = []
    for repeatNum in range(minSymbols, maxSymbols + 1):
        initList = list(product(varTransform, repeat=repeatNum))
        for e in initList:
            repeated = False
            for i in range(len(e)-1):
                if e[i] == e[i+1]:
                    repeated = True
            if e.count("P") > 1:
                repeated = True
            if not repeated:
                good.append(",".join(e))
    return good


def getVarTransformChoice(minSymbols=0, maxSymbols=2):
    nTsf = np.random.randint(minSymbols, maxSymbols+1)
    varTransform = ["P", "G", "D", "N"]
    good = []
    initList = list(product(varTransform, repeat=nTsf))
    for e in initList:
        repeated = False
        for i in range(len(e)-1):
            if e[i] == e[i+1]:
                repeated = True
        if e.count("P") > 1:
            repeated = True
        if not repeated:
            good.append(",".join(e))
    return np.random.choice(good)


def getBestVarTransform(n=30):
    best = ["G", "N,G,N,G", "", "N", "P,N,D", "P,N,D,N", "P,G", "P,N", "P,D", "P", "N,G", "G,N", "G,N,G,N", "N,G,N", "G,N,G", "P,D,N", "P,D,N,D", "N,P,D", "N,P,N,D", "N,P,G", "P,D,G", "D,N,D", "P,G,N", "P,N,G", "D,N,G", "P,G,N,G", "D", "P,N,P,D"]
    return best[:n]


def getNTreesChoice(minNum = 100, maxNum=5000):
    return np.random.randint(minNum, maxNum+1)


def getBoostType():
    old = ["AdaBoost", "RealAdaBoost", "Bagging", "AdaBoostR2", "Grad"]
    return ["Grad"]


def getShrinkageChoice(minNum = 0.0, maxNum=1.0):
    return np.round(np.random.uniform(minNum, maxNum), 5)


def getMaxDepthChoice(minNum = 4, maxNum=8):
    return np.random.randint(minNum, maxNum+1)


def getSeparationType():
    return ["CrossEntropy", "GiniIndex", "GiniIndexWithLaplace", "MisClassificationError", "SDivSqrtSPlusB", "RegressionVariance"]


def getPruneMethod():
    all =  ["NoPruning", "ExpectedError", "CostComplexity"]
    return ["ExpectedError", "CostComplexity"]


def getNCutsChoice(minNum = 1, maxNum=100):
    return np.random.randint(minNum, maxNum+1)


def getUseNvarsChoice(minNum = 1, maxNum=100):
    return np.random.randint(minNum, maxNum+1)


def getBaggedSampleFractionChoice(minNum = 0.1, maxNum=5.0):
    return np.round(np.random.uniform(minNum, maxNum), 5)


def getPruneStrengthChoice(minNum = 0, maxNum=100):
    return np.random.randint(minNum, maxNum+1)


def getPruningValFractionChoice(minNum = 0.0, maxNum=2.0):
    return np.round(np.random.uniform(minNum, maxNum), 5)

In [ ]:
m, com = getModels(50, letter="AB")

for i in m:
    print(i)

for i in com:
    print(i)

In [ ]:
def readModelsString():
    path = "tmva_models/models.txt"
    with open(path, 'r') as file:
        content = file.read()
        if len(content) > 0:
            return content
        raise Exception("Empty file: {}".format(path))


def readRMSsString(num = 0):
    path = "tmva_models/rms{}.txt".format(num)
    with open(path, 'r') as file:
        content = file.read()
        if len(content) > 0:
            return content
        raise Exception("Empty file: {}".format(path))


def getModelsInfo(models):
    infos = {}
    print(models.split(';')[0].split('"')[3].split(':'))
    for m in models.split(';')[:-1]:
        nameModel = m.split('"')[1][:-3]        
        optionsModel = []
        options = m.split('"')[3].split(':')
        if "VarTransform" in options[1]:
            optionsModel.append(options[1].split('=')[1])
            options = options[2:]
        else:
            optionsModel.append("")
            options = options[1:]
        optionsModel.append(int(options[0].split('=')[1]))
        optionsModel.append(options[1].split('=')[1])
        optionsModel.append(float(options[2].split('=')[1]))
        optionsModel.append(int(options[3].split('=')[1]))
        optionsModel.append(options[4].split('=')[1])
        optionsModel.append(int(options[5].split('=')[1]))
        optionsModel.append(options[6].split('=')[1])
        optionsModel.append(int(options[7].split('=')[1]))
        optionsModel.append(float(options[9].split('=')[1]))
        if options[10].split('=')[1] == "NoPruning":
            optionsModel.append(options[10].split('=')[1])
            optionsModel.append(int(0))
            optionsModel.append(float(0))
        else:
            optionsModel.append(options[10].split('=')[1])
            optionsModel.append(int(options[11].split('=')[1]))
            optionsModel.append(float(options[12].split('=')[1]))
        infos[nameModel] = optionsModel
    return infos


def fromDictToList(d):
    outList = []
    for key, value in d.items():
        outList.append([key] + value)
    return outList


def getModelsRMS(allRMS):
    allRMS = allRMS.replace(":", " ").replace("-", " ")
    RMS = {}
    for row in allRMS.split("\n"):
        #print(row.split())
        if len(row) > 0:
            RMS[row.split()[-8][:-2]] = float(row.split()[-5])
    return RMS


def concatenateInfoRMS(infos, rms0, rms1, rms2):
    agg = {}
    for k in infos.keys():
        agg[k] = infos[k] + [rms0[k]] + [rms1[k]] + [rms2[k]]
    return agg

In [ ]:
# Read models and convert to dictionary
mread = readModelsString()
infs = getModelsInfo(mread)
print(len(infs))
# Read RMS's of the three samples and convert to dictionary
sread0, sread1, sread2 = readRMSsString(0), readRMSsString(1), readRMSsString(2)
rms0, rms1, rms2 = getModelsRMS(sread0), getModelsRMS(sread1), getModelsRMS(sread2)
print(len(rms0), len(rms1), len(rms2))
# Aggregate models to rms
agg_list = fromDictToList(concatenateInfoRMS(infs, rms0, rms1, rms2))
print(len(agg_list), len(agg_list[0]))

In [ ]:
df = pd.DataFrame(agg_list, columns=["Model_name", "VarTransform", "NTrees", "BoostType", "Shrinkage", "MaxDepth", "SeparationType", "nCuts", "UseRandomisedTrees", "UseNvars", "BaggedFraction", "PruneMethod", "PruneStrength", "PruningValFraction", "RMS0", "RMS1", "RMS2"])
df["RMS"] = df[['RMS0', 'RMS1', 'RMS2']].mean(axis=1)

df = df[(df["RMS"].notna()) & (df["RMS"] > 0) & (df["RMS"] < 15) & (df["BoostType"] == "Grad") & (df["MaxDepth"] < 9) & (df["NTrees"] < 5000)]
df = df[(df["Shrinkage"] > 0.10) & (df["Shrinkage"] < 0.40)]
df = df[(df["BaggedFraction"] > 1.0) & (df["Shrinkage"] < 5.40)]
# Define intervals of continuous variables
intervals_Shrinkage = np.linspace(df["Shrinkage"].min(), df["Shrinkage"].max(), 16)
df['BinShrinkage'] = pd.cut(df['Shrinkage'], bins=intervals_Shrinkage, labels = (intervals_Shrinkage[:-1]+intervals_Shrinkage[1:])/2, include_lowest=True)
intervals_nCuts = np.linspace(df["nCuts"].min(), df["nCuts"].max(), 16)
df['BinnCuts'] = pd.cut(df['nCuts'], bins=intervals_nCuts, labels = (intervals_nCuts[:-1]+intervals_nCuts[1:])/2, include_lowest=True)
intervals_UseNvars = np.linspace(df["UseNvars"].min(), df["UseNvars"].max(), 16)
df['BinUseNvars'] = pd.cut(df['UseNvars'], bins=intervals_UseNvars, labels = (intervals_UseNvars[:-1]+intervals_UseNvars[1:])/2, include_lowest=True)
intervals_BaggedFraction = np.linspace(df["BaggedFraction"].min(), df["BaggedFraction"].max(), 16)
df['BinBaggedFraction'] = pd.cut(df['BaggedFraction'], bins=intervals_BaggedFraction, labels = (intervals_BaggedFraction[:-1]+intervals_BaggedFraction[1:])/2, include_lowest=True)
intervals_NTrees = np.linspace(df["NTrees"].min(), df["NTrees"].max(), 16)
df['BinNTrees'] = pd.cut(df['NTrees'], bins=intervals_NTrees, labels = (intervals_NTrees[:-1]+intervals_NTrees[1:])/2, include_lowest=True)
intervals_PruneStrength = np.linspace(df["PruneStrength"].min(), df["PruneStrength"].max(), 16)
df['BinPruneStrength'] = pd.cut(df['PruneStrength'], bins=intervals_PruneStrength, labels = (intervals_PruneStrength[:-1]+intervals_PruneStrength[1:])/2, include_lowest=True)
intervals_PruningValFraction = np.linspace(df["PruningValFraction"].min(), df["PruningValFraction"].max(), 16)
df['BinPruningValFraction'] = pd.cut(df['PruningValFraction'], bins=intervals_PruningValFraction, labels = (intervals_PruningValFraction[:-1]+intervals_PruningValFraction[1:])/2, include_lowest=True)

# Add metrics of the transformations
df["nTransf"] = df["VarTransform"].apply(lambda x: int((len(x)+1)/2))
df["firstTransf"] = df["VarTransform"].apply(lambda x: x[0] if len(x)>0 else 'X')
df["lastTransf"] = df["VarTransform"].apply(lambda x: x[-1] if len(x)>0 else 'X')


#df["RMS"] = df["RMS"].apply(lambda x: x-7.5)
#df = df[(df["PruneMethod"] != "NoPruning")]

display(df)

In [ ]:
plt.scatter(df["NTrees"], df["RMS"], s=10, color="#ff3333")
plt.errorbar(list(df.groupby('BinNTrees').groups.keys()), df.groupby('BinNTrees')['RMS'].mean().to_numpy(), df.groupby('BinNTrees')['RMS'].std().to_numpy(), fmt='-o', color="#3333ff", capsize=8)
plt.title("RMS vs NTrees")
#plt.yscale('log')
plt.grid()
plt.show()

plt.hist(df["NTrees"], bins=50)
plt.title("NTrees")
step_size = 100
plt.xticks(np.arange(0, 2000 + step_size, step_size))
plt.xticks(rotation=90)
plt.grid()
plt.show()


'''plt.scatter(df["BoostType"], df["RMS"], s=10, color="#ff3333")
plt.errorbar(list(df.groupby('BoostType').groups.keys()), df.groupby('BoostType')['RMS'].mean().to_numpy(), df.groupby('BoostType')['RMS'].std().to_numpy(), fmt='-o', color="#3333ff", capsize=8)
plt.title("RMS vs BoostType")
plt.grid()
plt.show()'''
'''
#print(df.groupby('BoostType')['RMS'].mean(), df.groupby('BoostType')['RMS'].std())
mean_bagging, mean_grad, std_bagging, std_grad = df.groupby('BoostType')['RMS'].mean()["Bagging"], df.groupby('BoostType')['RMS'].mean()["Grad"], df.groupby('BoostType')['RMS'].std()["Bagging"], df.groupby('BoostType')['RMS'].std()["Grad"]
#print(mean_bagging, mean_grad, std_bagging, std_grad)
plt.hist(df[df["BoostType"] == "Bagging"]["RMS"], bins=10, alpha=0.5, label='Bagging', density=True)
plt.hist(df[df["BoostType"] == "Grad"]["RMS"], bins=10, alpha=0.5, label='Grad', density=True)
x = np.linspace(7, 15, 100)
plt.plot(x, norm.pdf(x, mean_bagging, std_bagging), color='blue', label='Bagging')
plt.plot(x, norm.pdf(x, mean_grad, std_grad), color='orange', label='Grad')
plt.legend(loc='upper right')
plt.xlabel('RMS')
plt.ylabel('Frequency')
plt.title('Histogram of Two Distributions')
plt.grid()
plt.show()
print("Probability that RMS(Grad) < RMS(Bagging) = ", norm(mean_grad-mean_bagging, np.sqrt(std_bagging**2 + std_grad**2)).cdf(0.0))
'''
plt.scatter(df["Shrinkage"], df["RMS"], s=10, color="#ff3333")
plt.errorbar(list(df.groupby('BinShrinkage').groups.keys()), df.groupby('BinShrinkage')['RMS'].mean().to_numpy(), df.groupby('BinShrinkage')['RMS'].std().to_numpy(), fmt='-o', color="#3333ff", capsize=8)
plt.title("RMS vs Shrinkage")
#plt.yscale('log')
step_size = 0.025
plt.xticks(np.arange(0, 0.5 + step_size, step_size))
plt.xticks(rotation=90)
plt.grid()
plt.show()

plt.hist(df["Shrinkage"], bins=20)
plt.title("Shrinkage")
step_size = 0.1
plt.xticks(np.arange(0, 1 + step_size, step_size))
plt.xticks(rotation=90)
plt.grid()
plt.show()

plt.scatter(df["MaxDepth"], df["RMS"], s=10, color="#ff3333")
plt.errorbar(list(df.groupby('MaxDepth').groups.keys()), df.groupby('MaxDepth')['RMS'].mean().to_numpy(), df.groupby('MaxDepth')['RMS'].std().to_numpy(), fmt='-o', color="#3333ff", capsize=8)
plt.title("RMS vs MaxDepth")
#plt.yscale('log')
plt.grid()
plt.show()

'''plt.scatter(df["SeparationType"], df["RMS"], s=10, color="#ff3333")
plt.errorbar(list(df.groupby('SeparationType').groups.keys()), df.groupby('SeparationType')['RMS'].mean().to_numpy(), df.groupby('SeparationType')['RMS'].std().to_numpy(), fmt='o', color="#3333ff", capsize=8)
plt.title("RMS vs SeparationType")
plt.xticks(rotation=45)
plt.grid()
plt.show()'''

plt.scatter(df["nCuts"], df["RMS"], s=10, color="#ff3333")
plt.errorbar(list(df.groupby('BinnCuts').groups.keys()), df.groupby('BinnCuts')['RMS'].mean().to_numpy(), df.groupby('BinnCuts')['RMS'].std().to_numpy(), fmt='-o', color="#3333ff", capsize=8)
plt.title("RMS vs nCuts")
#plt.yscale('log')
plt.grid()
plt.show()

plt.hist(df["nCuts"], bins=50)
plt.title("nCuts")
step_size = 5
plt.xticks(np.arange(0, 100 + step_size, step_size))
plt.xticks(rotation=90)
plt.grid()
plt.show()

plt.scatter(df["UseNvars"], df["RMS"], s=10, color="#ff3333")
plt.errorbar(list(df.groupby('BinUseNvars').groups.keys()), df.groupby('BinUseNvars')['RMS'].mean().to_numpy(), df.groupby('BinUseNvars')['RMS'].std().to_numpy(), fmt='-o', color="#3333ff", capsize=8)
plt.title("RMS vs UseNvars")
#plt.yscale('log')
plt.grid()
plt.show()

plt.hist(df["UseNvars"], bins=50)
plt.title("UseNvars")
step_size = 5
plt.xticks(np.arange(0, 100 + step_size, step_size))
plt.xticks(rotation=90)
plt.grid()
plt.show()

plt.scatter(df["BaggedFraction"], df["RMS"], s=10, color="#ff3333")
plt.errorbar(list(df.groupby('BinBaggedFraction').groups.keys()), df.groupby('BinBaggedFraction')['RMS'].mean().to_numpy(), df.groupby('BinBaggedFraction')['RMS'].std().to_numpy(), fmt='-o', color="#3333ff", capsize=8)
plt.title("RMS vs BaggedFraction")
#plt.yscale('log')
plt.grid()
plt.show()

plt.hist(df["BaggedFraction"], bins=50)
plt.title("BaggedFraction")
step_size = 0.2
plt.xticks(np.arange(0, 5 + step_size, step_size))
plt.xticks(rotation=90)
plt.grid()
plt.show()


'''plt.scatter(df["nTransf"], df["RMS"], s=10, color="#ff3333")
plt.errorbar(list(df.groupby('nTransf').groups.keys()), df.groupby('nTransf')['RMS'].mean().to_numpy(), df.groupby('nTransf')['RMS'].std().to_numpy(), fmt='-o', color="#3333ff", capsize=8)
plt.title("RMS vs nTransf")
plt.grid()
plt.show()


plt.scatter(df["firstTransf"], df["RMS"], s=10, color="#ff3333")
plt.errorbar(list(df.groupby('firstTransf').groups.keys()), df.groupby('firstTransf')['RMS'].mean().to_numpy(), df.groupby('firstTransf')['RMS'].std().to_numpy(), fmt='o', color="#3333ff", capsize=8)
plt.title("RMS vs firstTransf")
plt.grid()
plt.show()


plt.scatter(df["lastTransf"], df["RMS"], s=10, color="#ff3333")
plt.errorbar(list(df.groupby('lastTransf').groups.keys()), df.groupby('lastTransf')['RMS'].mean().to_numpy(), df.groupby('lastTransf')['RMS'].std().to_numpy(), fmt='o', color="#3333ff", capsize=8)
plt.title("RMS vs lastTransf")
plt.grid()
plt.show()'''

'''plt.scatter(df["VarTransform"], df["RMS"], s=10, color="#ff3333")
plt.errorbar(list(df.groupby('VarTransform').groups.keys()), df.groupby('VarTransform')['RMS'].mean().to_numpy(), df.groupby('VarTransform')['RMS'].std().to_numpy(), fmt='o', color="#3333ff", capsize=8)
plt.title("RMS vs VarTransform")
plt.xticks(rotation=90)
plt.show()'''

plt.scatter(df["PruneMethod"], df["RMS"], s=10, color="#ff3333")
plt.errorbar(list(df.groupby('PruneMethod').groups.keys()), df.groupby('PruneMethod')['RMS'].mean().to_numpy(), df.groupby('PruneMethod')['RMS'].std().to_numpy(), fmt='o', color="#3333ff", capsize=8)
plt.title("RMS vs PruneMethod")
plt.xticks(rotation=45)
plt.grid()
plt.show()

plt.scatter(df["PruneStrength"], df["RMS"], s=10, color="#ff3333")
plt.errorbar(list(df.groupby('BinPruneStrength').groups.keys()), df.groupby('BinPruneStrength')['RMS'].mean().to_numpy(), df.groupby('BinPruneStrength')['RMS'].std().to_numpy(), fmt='-o', color="#3333ff", capsize=8)
plt.title("RMS vs PruneStrength")
#plt.yscale('log')
plt.grid()
plt.show()


plt.scatter(df["PruningValFraction"], df["RMS"], s=10, color="#ff3333")
plt.errorbar(list(df.groupby('BinPruningValFraction').groups.keys()), df.groupby('BinPruningValFraction')['RMS'].mean().to_numpy(), df.groupby('BinPruningValFraction')['RMS'].std().to_numpy(), fmt='-o', color="#3333ff", capsize=8)
plt.title("RMS vs PruningValFraction")
#plt.yscale('log')
plt.grid()
plt.show()



In [ ]:

print(df.loc[df['RMS'].idxmin()])